In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [ ]:
def convert_string_to_list(column):
    # return column.apply(lambda cell: np.fromstring(cell[1:-1], dtype=np.float64, sep=' '))
    return column.apply(lambda cell: np.array(literal_eval(cell)))

In [ ]:
def train_test_split(df, train_percent=.8):
    n_samples = df.shape[0]
    perm = np.random.permutation(n_samples)
    train_end = int(train_percent * n_samples)
    train = df.iloc[perm[:train_end]]
    test = df.iloc[perm[train_end:]]
    return train, test

In [ ]:
def read_csv(filename):
    df_samples = pd.read_csv(filename,sep=';',names=['V1','V2','V1V2','V2V1'])
    df_samples['V1'] = convert_string_to_list(df_samples['V1'])
    df_samples['V2'] = convert_string_to_list(df_samples['V2'])
    # df_samples.head()
    return df_samples

In [ ]:
def format_input(df_samples):
    df_input = df_samples.filter(['V1','V2','V1V2']).rename(columns={'V1': 'A', 'V2': 'B', 'V1V2': 'AB'})
    df_input = pd.concat([df_input,df_samples.filter(['V2','V1','V2V1']).rename(columns={'V2':'A', 'V1': 'B', 'V2V1': 'AB'})])
    return df_input

In [ ]:
def read_file_format_and_split(filename):
    df_samples = read_csv(filename)
    df_input = format_input(df_samples)
    return train_test_split(df_input)

In [ ]:
def save_model_json(model,config_name,dim,total):
    model_json = model.to_json()
    model_name = 'model_'+config_name+'_d'+str(dim)+'_s'+str(total)+'.json'
    with open(model_name,'w') as json_file:
        json_file.write(model_json)

In [ ]:
def save_model_weights(model,config_name,dim,total):
    model_weights_name = 'model_'+config_name+'_d'+str(dim)+'_s'+str(total)+'.h5'
    model.save_weights(model_weights_name)